In [ ]:
import redis

In [ ]:
# redis-py提供两个类Redis和StrictRedis用于实现Redis的命令
# StrictRedis用于实现大部分官方的命令，并使用官方的语法和命令（比如，SET命令对应与StrictRedis.set方法）
# Redis是StrictRedis的子类，用于向后兼容旧版本的redis-py
r = redis.Redis(host='127.0.0.1', port=30379, db=0, password=111111)
sr = redis.StrictRedis()

# 默认python3返回字节数组，python2返回字符串
r.set('a', 'b')
print(r.get('a'))
r.rpush('mylist', 'one')
r.rpop('mylist')

In [ ]:
# redis-py使用connection pool来管理对一个redis server的所有连接，避免每次建立、释放连接的开销
# 默认，每个Redis实例都会维护一个自己的连接池
# 可以直接建立一个连接池，然后作为参数Redis，这样就可以实现多个Redis实例共享一个连接池
shared_pool = redis.ConnectionPool(host='127.0.0.1', port=30379)

rp = redis.Redis(connection_pool=shared_pool)
rp.set('one', 'first')
rp.set('two', 'second')
print(rp.get('one'))
print(rp.get('two'))

In [ ]:
# 可以在一次请求中执行多个命令，这样避免了多次的往返时延。
pool = redis.ConnectionPool(host='127.0.0.1', port=9212)
rinstance = redis.Redis(connection_pool=pool)
pipe = rinstance.pipeline()

pipe.set('one', 'first')
pipe.set('two', 'second')
pipe.execute()

pipe.set('one', 'first').rpush('list', 'hello').rpush('list', 'world').execute()

In [ ]:
pool = redis.ConnectionPool(host='127.0.0.1', port=9212)
r = redis.Redis(connection_pool=pool)
# redis-py默认在一次pipeline中的操作是原子的，要改变这种方式，可以传入transaction=False
pipe_not_automic = r.pipeline(transaction=False)

In [ ]:
pool = redis.ConnectionPool(host='127.0.0.1', port=9212)
r1 = redis.Redis(connection_pool=pool)

try:
    pipe = r.pipeline()
    pipe.watch('k')

    pipe.set('k', '2')
    pipe.zadd('zk', 'aaa')
    pipe.execute()
except Exception as e:
    pipe.unwatch()
    print(e)